In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import mediapipe as mp

In [ ]:
DATA_DIR = './original_images'

In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [ ]:
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

In [ ]:
# data = []
# labels = []

# for dir_ in os.listdir(DATA_DIR):
#     for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
#         data_aux = []
#         img = cv2.imread(os.path.join(DATA_DIR, dir_, img_path))
#         img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#         results = hands.process(img_rgb)
#         if results.multi_hand_landmarks:
#             for hand_landmarks in results.multi_hand_landmarks:
#                 for i in range(len(hand_landmarks.landmark)):
#                     x = hand_landmarks.landmark[i].x
#                     y = hand_landmarks.landmark[i].y
#                     data_aux.append(x)
#                     data_aux.append(y)
            
#             data.append(data_aux)
#             labels.append(dir_)

        

In [ ]:
import numpy as np

In [ ]:
data = []
labels = []

# Set the number of images to process from each subfolder
num_images_to_process = 1000
num_landmarks = 21  # Assuming 21 landmarks per hand

for dir_ in os.listdir(DATA_DIR):
    image_count = 0  # Initialize image counter for each subfolder
    for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
        if image_count >= num_images_to_process:
            break  # Stop processing if we have processed 10 images

        data_aux = []
        img = cv2.imread(os.path.join(DATA_DIR, dir_, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            # Only consider the first hand detected
            hand_landmarks = results.multi_hand_landmarks[0]  # Get the first hand

            x = [0] * num_landmarks
            y = [0] * num_landmarks

            for i in range(min(len(hand_landmarks.landmark), num_landmarks)):
                x[i] = hand_landmarks.landmark[i].x
                y[i] = hand_landmarks.landmark[i].y

            data_aux.extend(x)
            data_aux.extend(y)
        else:
            # If no landmarks detected, fill with zeros
            data_aux = [0] * (2 * num_landmarks)  # Same length as if landmarks were detected

        # Append data only if it has the expected length
        if len(data_aux) == 2 * num_landmarks:  # Ensure the correct length
            data.append(data_aux)
            labels.append(dir_)
        else:
            print(f"Inconsistent data length in {img_path}: {len(data_aux)}")

        image_count += 1  # Increment image counter

# Convert to numpy arrays after ensuring all data_aux are consistent in length



In [ ]:
data = np.array(data)
labels = np.array(labels)

In [ ]:
import pickle
import numpy as np

In [ ]:
with open("data_temp", "wb") as fp:   #Pickling
    pickle.dump(data, fp)
with open("labels_temp", "wb") as fp:   #Pickling
    pickle.dump(labels, fp)

In [ ]:
with open("data_temp", "rb") as fp:   # Unpickling
    data = pickle.load(fp)
with open("labels_temp", "rb") as fp:   # Unpickling
    labels = pickle.load(fp)

In [ ]:
pip install scikit-learn

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

with open("labels_encoded", "wb") as fp:   #Pickling
    pickle.dump(labels_encoded, fp)

# isko tensorflow se run krna
# labels_categorical = to_categorical(labels_encoded)

In [ ]:
with open("labels_encoded", "rb") as fp:   # Unpickling
   labels_encoded = pickle.load(fp)

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
labels_categorical = to_categorical(labels_encoded)

In [ ]:
with open("labels_categorical", "wb") as fp:   #Pickling
    pickle.dump(labels_categorical, fp)

In [ ]:
with open("labels_categorical", "rb") as fp:   # Unpickling
   labels_categorical = pickle.load(fp)

In [ ]:
x_train,x_test, y_train, y_test = train_test_split(data, labels_categorical, test_size=0.2, shuffle=True, stratify=labels)

In [ ]:
with open("x_train", "wb") as fp:   #Pickling
    pickle.dump(x_train, fp)
with open("x_test", "wb") as fp:   #Pickling
    pickle.dump(x_test, fp)

with open("y_train", "wb") as fp:   #Pickling
    pickle.dump(y_train, fp)
with open("y_test", "wb") as fp:   #Pickling
    pickle.dump(y_test, fp)


In [ ]:
model = RandomForestClassifier()

In [ ]:
model.fit(x_train, y_train)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
accuracy_score(y_pred, y_test)

In [ ]:
# New Models Now

In [1]:
import pickle
import numpy as np

with open("x_train", "rb") as fp:   # Unpickling
    x_train = pickle.load(fp)
with open("x_test", "rb") as fp:   # Unpickling
    x_test = pickle.load(fp)

with open("y_train", "rb") as fp:   # Unpickling
    y_train = pickle.load(fp)
with open("y_test", "rb") as fp:   # Unpickling
    y_test = pickle.load(fp)

with open("labels_categorical", "rb") as fp:   # Unpickling
    labels_categorical = pickle.load(fp)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization, Normalization

In [ ]:
normalizer = Normalization()
normalizer.adapt(x_train)
x_train_norm = normalizer(x_train)
# y_train_norm = normalizer(y_train)


# normalizer.adapt(x_train)
# x_train_norm = normalizer(x_train)

# normalizer.adapt(x_train)
# x_train_norm = normalizer(x_train)

model = Sequential([
    Dense(256, input_dim=42, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(labels_categorical.shape[1], activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.2)

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_accuracy:.2f}')

In [ ]:
pip install mediapipe==0.9.1.0

In [6]:
pip list

Package               Version
--------------------- -----------
absl-py               2.1.0
accelerate            0.33.0
aiohappyeyeballs      2.4.0
aiohttp               3.10.5
aiosignal             1.3.1
asttokens             2.4.1
async-timeout         4.0.3
attrs                 24.2.0
certifi               2024.8.30
cffi                  1.17.0
charset-normalizer    3.3.2
colorama              0.4.6
comm                  0.2.2
contourpy             1.3.0
cycler                0.12.1
datasets              2.21.0
debugpy               1.8.5
decorator             5.1.1
dill                  0.3.8
exceptiongroup        1.2.2
executing             2.0.1
filelock              3.15.4
flatbuffers           24.3.25
fonttools             4.53.1
frozenlist            1.4.1
fsspec                2024.6.1
huggingface-hub       0.24.6
idna                  3.8
ipykernel             6.29.5
ipython               8.27.0
jax                   0.4.31
jaxlib                0.4.31
jedi                


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip list

Package                      Version
---------------------------- -----------
absl-py                      2.1.0
aiohappyeyeballs             2.3.5
aiohttp                      3.10.2
aiosignal                    1.3.1
asttokens                    2.4.1
astunparse                   1.6.3
async-timeout                4.0.3
attrs                        24.2.0
beautifulsoup4               4.12.3
cachetools                   5.4.0
certifi                      2024.7.4
cffi                         1.17.0
charset-normalizer           3.3.2
cloudpickle                  3.0.0
colorama                     0.4.6
comm                         0.2.2
contourpy                    1.2.1
cycler                       0.12.1
debugpy                      1.8.2
decorator                    5.1.1
discord.py                   2.4.0
exceptiongroup               1.2.2
executing                    2.0.1
filelock                     3.15.4
flatbuffers                  24.3.25
fonttools                    4.53.1


In [2]:
import xgboost as xgb

In [3]:
model = xgb.XGBClassifier(objective='multi:softmax', num_class=36, eval_metric='mlogloss')

In [4]:
import numpy as np

# Convert one-hot encoded labels to integer labels
y_train_int = np.argmax(y_train, axis=1)
y_test_int = np.argmax(y_test, axis=1)

In [5]:
model.fit(x_train, y_train_int)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_class=36, num_parallel_tree=None, ...)

In [6]:
from sklearn.metrics import accuracy_score

y_pred = model.predict(x_test)

# Evaluate
accuracy = accuracy_score(y_test_int, y_pred)

In [7]:
accuracy

0.6723611111111111

In [26]:
import xgboost as xgb
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
import cv2
import mediapipe as mp
from sklearn.ensemble import RandomForestClassifier

In [27]:



data = []
labels = []

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,  # You can detect up to 2 hands.
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
)

# Set the number of images to process from each subfolder
num_images_to_process = 1000
num_landmarks = 21  # Assuming 21 landmarks per hand
DATA_DIR = './original_images'

for dir_ in os.listdir(DATA_DIR)[:5]:
    image_count = 0  # Initialize image counter for each subfolder
    for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
        if image_count >= num_images_to_process:
            break  # Stop processing if we have processed 10 images

        data_aux = []
        img = cv2.imread(os.path.join(DATA_DIR, dir_, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            # Only consider the first hand detected
            hand_landmarks = results.multi_hand_landmarks[0]  # Get the first hand

            x = [0] * num_landmarks
            y = [0] * num_landmarks

            for i in range(min(len(hand_landmarks.landmark), num_landmarks)):
                x[i] = hand_landmarks.landmark[i].x
                y[i] = hand_landmarks.landmark[i].y

            data_aux.extend(x)
            data_aux.extend(y)
        else:
            # If no landmarks detected, fill with zeros
            data_aux = [0] * (2 * num_landmarks)  # Same length as if landmarks were detected

        # Append data only if it has the expected length
        if len(data_aux) == 2 * num_landmarks:  # Ensure the correct length
            data.append(data_aux)
            labels.append(dir_)
        else:
            print(f"Inconsistent data length in {img_path}: {len(data_aux)}")

        image_count += 1  # Increment image counter

# Convert to numpy arrays after ensuring all data_aux are consistent in length

data = np.array(data)
labels = np.array(labels)

c:\Users\rudra\Desktop\stuff\DS and ML\DL\Sign Lang Final(from sign language model)\venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [28]:
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [30]:

model = xgb.XGBClassifier(objective='multi:softmax', num_class=36, eval_metric='mlogloss')

# Convert one-hot encoded labels to integer labels

# y_test_int = np.argmax(y_test, axis=1)

In [32]:
model.fit(data, labels_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_class=36, num_parallel_tree=None, ...)

In [33]:


rf_model = RandomForestClassifier(
    n_estimators=500,  # A larger number of trees can help with robustness
    max_depth=None,  # Let trees grow until all leaves are pure or until all leaves contain fewer than min_samples_split samples
    min_samples_split=10,  # Reduce overfitting by requiring at least 10 samples to split a node
    min_samples_leaf=5,  # Further reduce overfitting by requiring at least 5 samples at a leaf node
    max_features='sqrt',  # Square root of the number of features; generally a good choice for classification tasks
    random_state=42,  # For reproducibility
)

# Train the model
rf_model.fit(data, labels_encoded)

RandomForestClassifier(min_samples_leaf=5, min_samples_split=5,
                       n_estimators=500, random_state=42)

In [1]:
import pickle
with open("model.p", "rb") as fp:   # Unpickling
    model = pickle.load(fp)

In [7]:

data = './original_images'
prediction_class = list()
for label in os.listdir(data)[:5]:
    prediction_class.append(label)

In [34]:
# Initialize MediaPipe Hands.
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,  # You can detect up to 2 hands.
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
)



# Initialize MediaPipe drawing tools.
mp_drawing = mp.solutions.drawing_utils

# Start video capture from the webcam.
cap = cv2.VideoCapture(0)
num_landmarks = 21
labels = dict()
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Ignoring empty camera frame.")
        continue
    
    H, W, _ = frame.shape
    # Convert the BGR frame to RGB for MediaPipe.
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and get hand landmarks.
    result = hands.process(frame_rgb)

    # Draw landmarks on the frame.
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
            )

        data_aux = []
        x_ = []
        y_ = []
        if result.multi_hand_landmarks:
            # Only consider the first hand detected
            hand_landmarks = result.multi_hand_landmarks[0]  # Get the first hand

            x = [0] * num_landmarks
            y = [0] * num_landmarks

            for i in range(min(len(hand_landmarks.landmark), num_landmarks)):
                x[i] = hand_landmarks.landmark[i].x
                y[i] = hand_landmarks.landmark[i].y

            data_aux.extend(x)
            data_aux.extend(y)
            x_.append(x)
            y_.append(y)
        else:
            # If no landmarks detected, fill with zeros
            data_aux = [0] * (2 * num_landmarks)  # Same length as if landmarks were detected

        # Append data only if it has the expected length
        prediction = rf_model.predict([np.asarray(data_aux)])
        # print(prediction_class[prediction[0]])
    
        x1 = int(min(x_[0]) * W)
        y1 = int(min(y_[0]) * H)
        

        x2 = int(max(x_[0]) * W)
        y2 = int(max(y_[0]) * H)
        
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,0,0), 4)
        cv2.putText(frame, prediction_class[prediction[0]], (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0,0,0), 3, cv2.LINE_AA)

    # Display the frame with hand landmarks.
    cv2.imshow('Hand Landmarks', frame)

    # Press 'q' to exit the loop.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources.
cap.release()
cv2.destroyAllWindows()
hands.close()


c:\Users\rudra\Desktop\stuff\DS and ML\DL\Sign Lang Final(from sign language model)\venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\rudra\Desktop\stuff\DS and ML\DL\Sign Lang Final(from sign language model)\venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\rudra\Desktop\stuff\DS and ML\DL\Sign Lang Final(from sign language model)\venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMess

In [15]:
cap.release() 
# Destroy all the windows 
cv2.destroyAllWindows()

In [2]:
pip freeze

absl-py==2.1.0Note: you may need to restart the kernel to use updated packages.

aiohappyeyeballs==2.3.5
aiohttp==3.10.2
aiosignal==1.3.1
asttokens==2.4.1
astunparse==1.6.3
async-timeout==4.0.3
attrs==24.2.0
beautifulsoup4==4.12.3
cachetools==5.4.0
certifi==2024.7.4
cffi==1.17.0
charset-normalizer==3.3.2
cloudpickle==3.0.0
colorama==0.4.6
comm==0.2.2
contourpy==1.2.1
cycler==0.12.1
debugpy==1.8.2
decorator==5.1.1
discord.py==2.4.0
exceptiongroup==1.2.2
executing==2.0.1
filelock==3.15.4
flatbuffers==24.3.25
fonttools==4.53.1
frozenlist==1.4.1
gast==0.4.0
gdown==5.2.0
google-auth==2.32.0
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
grpcio==1.65.1
gym==0.26.2
gym-notices==0.0.8
h5py==3.11.0
idna==3.7
image-classifiers @ git+https://github.com/qubvel/classification_models.git@a0f006e05485a34ccf871c421279864b0ccd220b
imageio==2.34.2
ipykernel==6.29.5
ipython==8.26.0
jax==0.4.31
jaxlib==0.4.31
jedi==0.19.1
joblib==1.4.2
jupyter_client==8.6.2
jupyter_core==5.7.2
keras==2.10.0
Keras-Applica

In [5]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip list

Package                      Version
---------------------------- -----------
absl-py                      2.1.0
aiohappyeyeballs             2.3.5
aiohttp                      3.10.2
aiosignal                    1.3.1
asttokens                    2.4.1
astunparse                   1.6.3
async-timeout                4.0.3
attrs                        24.2.0
beautifulsoup4               4.12.3
cachetools                   5.4.0
certifi                      2024.7.4
cffi                         1.17.0
charset-normalizer           3.3.2
cloudpickle                  3.0.0
colorama                     0.4.6
comm                         0.2.2
contourpy                    1.2.1
cycler                       0.12.1
debugpy                      1.8.2
decorator                    5.1.1
discord.py                   2.4.0
exceptiongroup               1.2.2
executing                    2.0.1
filelock                     3.15.4
flatbuffers                  24.3.25
fonttools                    4.53.1
